# Big query sql

In [40]:
%cd drive/My\ Drive/Colab\ Notebooks/DLG/slack_report/slack-report-2020

/content/drive/My Drive/Colab Notebooks/DLG/slack_report/slack-report-2020


In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from getpass import getpass
pjt_code = getpass('ProjectCode: ')

ProjectCode: ··········


In [9]:
%%bigquery --project {pjt_code} df_reply_table
DECLARE start_date DATE DEFAULT "2019-07-01";
DECLARE end_date DATE DEFAULT "2020-06-30";

WITH 
-- get all talk data with filling the null thread_ts as ts
all_talks_with_filled_thread_ts AS (
  SELECT
    channel_id, ts AS thread_ts, ts, talk_user, talk_id
  FROM
    mgm.talk
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    thread_ts IS NULL AND -- fill null thread_ts as ts
    talk_user IS NOT NULL -- exclude bot talks
  UNION ALL
  SELECT
    channel_id, thread_ts, ts, talk_user, talk_id
  FROM
    mgm.talk
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    thread_ts IS NOT NULL AND -- if thread_ts is not null no replace
    talk_user IS NOT NULL -- exclude bot talks
),
-- get first talk ts in all threads
-- mainly first talk ts = thread_ts but in some case not same
first_talk_in_threads AS (
  SELECT
    channel_id, thread_ts, MIN(ts) AS ts -- get MIN(ts) because some first talk ts > thread_ts
  FROM
    all_talks_with_filled_thread_ts
  GROUP BY
    channel_id, thread_ts
  ORDER BY thread_ts
),
-- join the talk_user, talk_id for first talks
thread_first_user_master AS (
  SELECT
    f.channel_id, f.thread_ts, f.ts, a.talk_user, a.talk_id
  FROM
    first_talk_in_threads f
      LEFT JOIN
        all_talks_with_filled_thread_ts a
      ON
        f.channel_id = a.channel_id AND f.thread_ts = a.thread_ts AND f.ts = a.ts
),
-- get the reply talks (non first talks) on each threads
reply_talk_data AS (
  SELECT
    DISTINCT channel_id, thread_ts, ts, talk_user, talk_id, target_date
  FROM
    mgm.talk AS raw
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    talk_user IS NOT NULL AND
    NOT EXISTS(
      SELECT talk_id
      FROM thread_first_user_master AS mstr
      WHERE raw.talk_id = mstr.talk_id
    )
  ORDER BY thread_ts
)

SELECT
  rp.channel_id,
  ROW_NUMBER() OVER(PARTITION BY rp.channel_id, rp.thread_ts ORDER BY rp.thread_ts, rp.ts, rp.channel_id) AS reply_num,
  rp.ts AS reply_ts, rp.talk_id AS reply_id, rp.talk_user AS reply_user,
  tm.thread_ts, tm.talk_id AS talk_id, tm.talk_user AS talk_user, rp.target_date
FROM 
  reply_talk_data rp
    LEFT JOIN
      thread_first_user_master tm
    ON
      rp.channel_id = tm.channel_id AND rp.thread_ts = tm.thread_ts
ORDER BY
  rp.thread_ts, rp.ts, rp.channel_id
;

In [10]:
df_reply_table

,channel_id,reply_num,reply_ts,reply_id,reply_user,thread_ts,talk_id,talk_user,target_date
0,CJR0XDM1D,1,2019-06-30 06:14:09.124500+00:00,8ae852c7-220d-49a7-b901-506b2ab541da,UJRAL005U,2019-06-24 05:11:47.002700+00:00,21ad0d2a-1a84-4e26-b80d-9d3a59be67b2,UJRAL005U,2019-07-01
1,CJR0XDM1D,2,2019-06-30 09:01:35.137800+00:00,8838d72d-6b66-4d8f-b088-b2099bc5b8a3,UKT5BQD3P,2019-06-24 05:11:47.002700+00:00,21ad0d2a-1a84-4e26-b80d-9d3a59be67b2,UJRAL005U,2019-07-01
2,CJR0XDM1D,3,2019-06-30 09:01:49.138500+00:00,44651094-0ad1-48ca-8655-c1c6455dea51,UKT5BQD3P,2019-06-24 05:11:47.002700+00:00,21ad0d2a-1a84-4e26-b80d-9d3a59be67b2,UJRAL005U,2019-07-01
3,CJR0XDM1D,4,2019-06-30 09:51:28.141200+00:00,225f8cc5-4352-4733-9068-29a08d6a26bf,UJRAL005U,2019-06-24 05:11:47.002700+00:00,21ad0d2a-1a84-4e26-b80d-9d3a59be67b2,UJRAL005U,2019-07-01
4,CJR0XDM1D,5,2019-06-30 09:54:25.145400+00:00,69dd2180-8090-4ee0-b8c4-f50b740e0066,UJRAL005U,2019-06-24 05:11:47.002700+00:00,21ad0d2a-1a84-4e26-b80d-9d3a59be67b2,UJRAL005U,2019-07-01
...,...,...,...,...,...,...,...,...,...
17208,CRYJLV9JP,2,2020-06-29 11:56:41.015600+00:00,2369500a-5f73-4061-bc7a-54373785aae1,UN1V4ELKV,2020-06-29 11:33:39.014900+00:00,44535bbb-e325-4900-9429-b7d6227aa85e,UN1V4ELKV,2020-06-30
17209,CRYJLV9JP,3,2020-06-29 11:57:41.015800+00:00,e1e5ff60-e6cd-47c6-9095-334260de25e6,U01247C09MH,2020-06-29 11:33:39.014900+00:00,44535bbb-e325-4900-9429-b7d6227aa85e,UN1V4ELKV,2020-06-30
17210,CRYJLV9JP,1,2020-06-29 13:40:34.018000+00:00,c6560478-fd53-4b49-87d5-e5915209c364,U01247C09MH,2020-06-29 13:37:58.017900+00:00,fae178a7-1ea8-49ec-ab7e-0357ef097288,UN1V4ELKV,2020-06-30
17211,CRYJLV9JP,2,2020-06-29 13:42:47.018200+00:00,762efaef-3573-4352-b8e3-207f12f6cbba,UN1V4ELKV,2020-06-29 13:37:58.017900+00:00,fae178a7-1ea8-49ec-ab7e-0357ef097288,UN1V4ELKV,2020-06-30


In [8]:
import pandas as pd
df.to_csv('reply_table_20190701-20200630.csv', index=False)

In [ ]:
%%bigquery --project {pjt_code} df_user_master
DECLARE start_date DATE DEFAULT "2019-06-01";

-- 各ユーザIDごとの最新日を取得
WITH user_max_trg_dates AS (
  SELECT
    user_id, MAX(target_date) AS max_target_date
  FROM
    mgm.user
  WHERE target_date > start_date
  GROUP BY user_id
),
-- 各ユーザIDごとの発言数
user_talk_counts AS (
  SELECT
    talk_user, COUNT(DISTINCT talk_id) AS talk_count
  FROM
    mgm.talk
  WHERE target_date > start_date
  GROUP BY talk_user
),
-- 最新日とトーク数をマージ
user_master_data AS(
  SELECT
    m.user_id, m.max_target_date, c.talk_count
    FROM user_max_trg_dates m
      JOIN user_talk_counts c
      ON m.user_id = c.talk_user
),

-- userテーブルに一部日付において重複があるので、それを消去（JOIN時に邪魔）
all_distinct_table AS(
  SELECT
    DISTINCT user_id, real_name_normalized, display_name_normalized, target_date
  FROM
    mgm.user
  WHERE
    target_date > start_date
),
-- 各target_date毎のuser_idとname
-- display_name=nullの方もいるので、その場合はreal_nameを使用
name_normalized_table AS(
  SELECT
    user_id, real_name_normalized AS name, target_date
  FROM
    all_distinct_table
  WHERE
    display_name_normalized IS NULL
  UNION ALL
  SELECT
    user_id, display_name_normalized AS name, target_date
  FROM
    all_distinct_table
  WHERE
    display_name_normalized IS NOT NULL
  ORDER BY target_date, user_id
)
-- 各ユーザの最新日にて該当日のnameをJOIN
SELECT
  m.user_id, u.name, u.target_date, m.talk_count
FROM user_master_data m
  INNER JOIN
    name_normalized_table u
  ON
    m.user_id = u.user_id AND m.max_target_date = u.target_date
ORDER BY m.max_target_date, m.user_id
;


In [ ]:
df_value_count = df_user_master['name'].value_counts().to_frame()
double_name = df_value_count[df_value_count['name']>1].index.tolist()

In [ ]:
for dn in double_name:
  df_tmp = df_user_master[df_user_master['name']==dn]

In [ ]:
df_user_master.to_csv('user_master.csv', index=False)

In [ ]:
df_user_master.head()

,user_id,name,target_date,talk_count,count
0,UKFNDDER1,こんちゃん,2020-04-03,108,1
1,UKLQEKEV7,本間 翔太,2020-04-03,22,1
2,UKP483XHR,u950527,2020-04-03,9,1
3,UKP4HTQ1G,Naoki,2020-04-03,23,1
4,UKPCQAQLA,鈴木淳平,2020-04-03,29,1


In [ ]:
df_user_master['count'] = 1
df_user_master.groupby(['user_id', 'max_target_date']).count().max()

KeyError: ignored

In [ ]:
df_user_master

,user_id,max_target_date,count
0,UKG6BDAPL,2020-06-16,1
1,UM9G63GUR,2020-05-11,1
2,UPAD4C0B0,2020-04-03,1
3,UTKK6HK2B,2020-07-12,1
4,U012PJ9KJNS,2020-07-12,1
...,...,...,...
405,URDDX224S,2020-07-12,1
406,URPSGF5SP,2020-07-12,1
407,UR126F540,2020-06-09,1
408,URDDG8EVA,2020-07-12,1


In [ ]:
%%bigquery --project {pjt_code} df_test
DECLARE start_date DATE DEFAULT "2019-06-01";
WITH distinct_table AS(
  SELECT
    DISTINCT user_id, real_name_normalized, display_name_normalized, target_date
  FROM
    mgm.user
  WHERE
    target_date > start_date
)
SELECT
  user_id, real_name_normalized AS name, target_date
FROM
  distinct_table
WHERE
  display_name_normalized IS NULL
UNION ALL
SELECT
  user_id, display_name_normalized AS name, target_date
FROM
  distinct_table
WHERE
  display_name_normalized IS NOT NULL
ORDER BY target_date, user_id

In [ ]:
df_test

,user_id,name,target_date
0,UJKFAPBCJ,岡村龍弥,2019-06-24
1,UJRAL005U,村上智之 ｷﾞﾙﾄﾞ代表 筋ﾄﾚで学ぶﾃﾞｰﾀ分析,2019-06-24
2,UJRDMQSAD,勝又健太 kenta.katsumata,2019-06-24
3,UKEMYD9R8,tetsuroito,2019-06-24
4,UKFLR3VHR,相川 仁,2019-06-24
...,...,...,...
86513,UUAUDBR1N,戸嶋 龍哉,2020-07-12
86514,UUMB12SGZ,永田ゆかり Yukari Nagata,2020-07-12
86515,UV1GRG3UZ,Kazuhii,2020-07-12
86516,UV1J8NN6B,funakoshi,2020-07-12


In [ ]:
df_test.groupby(['user_id', 'target_date']).count().max()

real_name_normalized       1
display_name_normalized    1
dtype: int64

In [ ]:
%%bigquery --project {pjt_code} df_test
DECLARE start_date DATE DEFAULT "2019-06-01";
SELECT DISTINCT *
FROM mgm.user
WHERE target_date > start_date


In [ ]:
df_test

,user_id,real_name_normalized,display_name_normalized,target_date
0,URDT39KGV,後藤,None,2020-05-03
1,U012JJPCQP5,yoshizirou,None,2020-05-03
2,U012WSURE92,RAI,RAI,2020-05-03
3,UV1GRG3UZ,Kazuhii,Kazuhii,2020-05-03
4,UTK77QX52,tobi,SHIMADA,2020-05-03
...,...,...,...,...
86513,UNLNYN8VA,杉本 光一,None,2019-10-13
86514,UMFDK5ETW,Y,Y,2019-10-13
86515,ULM6FBU11,tomo.h,tomo.h,2019-10-13
86516,UMJ5SQSCF,Keisuke Sugiyama,Keisuke Sugiyama,2019-10-13


In [ ]:
%%bigquery --project {pjt_code} df_test
DECLARE start_date DATE DEFAULT "2019-06-01";

-- 各ユーザIDごとの最新日を取得
WITH user_max_trg_dates AS (
  SELECT
    user_id, MAX(target_date) AS max_target_date
  FROM
    mgm.user
  WHERE target_date > start_date
  GROUP BY user_id
),
-- 各ユーザIDごとの発言数
user_talk_counts AS (
  SELECT
    talk_user, COUNT(DISTINCT talk_id) AS talk_count
  FROM
    mgm.talk
  WHERE target_date > start_date
  GROUP BY talk_user
)
-- 最新日とトーク数をマージ
  SELECT
    m.user_id, m.max_target_date, c.talk_count
    FROM user_max_trg_dates m
      JOIN user_talk_counts c
      ON m.user_id = c.talk_user

In [ ]:
df_test

,user_id,max_target_date,talk_count
0,UML7CC4N8,2020-04-03,9
1,UMRST297C,2020-07-13,5
2,UN13XU726,2020-04-03,1
3,UM6RV9SP4,2020-04-03,21
4,UTRSFL2LS,2020-07-13,53
...,...,...,...
360,URDDX224S,2020-07-13,721
361,URPSGF5SP,2020-07-13,30
362,UR126F540,2020-06-09,96
363,URDDG8EVA,2020-07-13,116


# 最も盛り上がったスレッドを取得

In [16]:
df_reply_ranking = df_reply_table.groupby(['channel_id','talk_id']).max()['reply_num'].reset_index(drop = False).sort_values(by='reply_num', ascending=False)

In [17]:
df_reply_ranking.head(10)

,channel_id,talk_id,reply_num
1110,CJR0XDM1D,99f10507-a382-4e48-b5c0-ab61cc8ad16f,176
1202,CJR0XDM1D,c2acbbfb-297f-4ed3-adb7-1d13b5702466,144
460,CJNKJ8JKW,2893fde6-b60a-4cd1-a70e-8731e2a47e4a,124
364,CJCNV9LG2,be328e6a-27fc-4068-bee7-e0f42b2afa65,108
379,CJCNV9LG2,d3127fb1-d943-4cf6-8bb4-c09c05acf9c5,107
290,CJCNV9LG2,6cb23741-5c6a-4f5b-8332-0dd4d1ed31d5,76
1434,CL4V80D40,e7aa4eb4-4c15-4095-9d23-94d1c620e2de,74
361,CJCNV9LG2,bc8ac552-17ea-42a3-8620-24ace5df3605,74
1095,CJR0XDM1D,949584cf-5bbe-4ff0-b1d7-ae0b26c8b924,72
904,CJR0XDM1D,380549d4-649d-4d81-b918-cbd126bcd640,71


In [18]:
id = '99f10507-a382-4e48-b5c0-ab61cc8ad16f'
df_reply_table[df_reply_table['talk_id']==id]

,channel_id,reply_num,reply_ts,reply_id,reply_user,thread_ts,talk_id,talk_user,target_date
7457,CJR0XDM1D,1,2020-01-21 01:29:24.070700+00:00,2f0f342d-8fcd-43f5-94c8-3d05240df2a3,UJRAL005U,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-22
7458,CJR0XDM1D,2,2020-01-21 01:33:38.070900+00:00,19e42145-dc5b-46f7-9ea6-a10dc9e8c0c2,UMG0947NY,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-22
7459,CJR0XDM1D,3,2020-01-21 01:36:54.071200+00:00,fefb3942-c790-47e2-8d65-e0dc145953bd,UJRAL005U,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-22
7460,CJR0XDM1D,4,2020-01-21 01:42:41.071400+00:00,dbdc9077-b4e0-4a28-b8f2-41bc045ed03e,UMG0947NY,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-22
7461,CJR0XDM1D,5,2020-01-21 01:43:42.071600+00:00,19620a7b-b063-4414-80f6-6694d197a422,UJRAL005U,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-22
...,...,...,...,...,...,...,...,...,...
7628,CJR0XDM1D,172,2020-01-27 13:04:34.093500+00:00,adccdb29-e461-4ce1-afc4-60046d92c1eb,UMTMKBB1Q,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-28
7629,CJR0XDM1D,173,2020-01-27 13:05:01.093800+00:00,6984a71d-8fbe-40de-a72d-d94a6ffb252c,UJRAL005U,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-28
7630,CJR0XDM1D,174,2020-01-27 13:05:10.094100+00:00,f1574ca1-ac3d-4072-a163-a85a4874fa54,UL2TY2ERL,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-28
7631,CJR0XDM1D,175,2020-01-27 13:05:17.094300+00:00,43f53ba9-41cd-49e9-990e-d5c608b19abb,US69FTTH6,2020-01-21 01:25:52.070600+00:00,99f10507-a382-4e48-b5c0-ab61cc8ad16f,UMG0947NY,2020-01-28


In [ ]:
%%bigquery --project {pjt_code} df_longest_reply
DECLARE start_date DATE DEFAULT "2019-07-01";
DECLARE end_date DATE DEFAULT "2020-06-30";
DECLARE target_talk_id STRING DEFAULT "2893fde6-b60a-4cd1-a70e-8731e2a47e4a";


WITH 
-- ここから、reply_tableの作成
-- get all talk data with filling the null thread_ts as ts
all_talks_with_filled_thread_ts AS (
  SELECT
    channel_id, ts AS thread_ts, ts, talk_user, talk_id
  FROM
    mgm.talk
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    thread_ts IS NULL AND -- fill null thread_ts as ts
    talk_user IS NOT NULL -- exclude bot talks
  UNION ALL
  SELECT
    channel_id, thread_ts, ts, talk_user, talk_id
  FROM
    mgm.talk
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    thread_ts IS NOT NULL AND -- if thread_ts is not null no replace
    talk_user IS NOT NULL -- exclude bot talks
),
-- get first talk ts in all threads
-- mainly first talk ts = thread_ts but in some case not same
first_talk_in_threads AS (
  SELECT
    channel_id, thread_ts, MIN(ts) AS ts -- get MIN(ts) because some first talk ts > thread_ts
  FROM
    all_talks_with_filled_thread_ts
  GROUP BY
    channel_id, thread_ts
  ORDER BY thread_ts
),
-- join the talk_user, talk_id for first talks
thread_first_user_master AS (
  SELECT
    f.channel_id, f.thread_ts, f.ts, a.talk_user, a.talk_id
  FROM
    first_talk_in_threads f
      LEFT JOIN
        all_talks_with_filled_thread_ts a
      ON
        f.channel_id = a.channel_id AND f.thread_ts = a.thread_ts AND f.ts = a.ts
),
-- get the reply talks (non first talks) on each threads
reply_talk_data AS (
  SELECT
    DISTINCT channel_id, thread_ts, ts, talk_user, talk_id, target_date
  FROM
    mgm.talk AS raw
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    talk_user IS NOT NULL AND
    NOT EXISTS(
      SELECT talk_id
      FROM thread_first_user_master AS mstr
      WHERE raw.talk_id = mstr.talk_id
    )
  ORDER BY thread_ts
),
reply_table AS (
  SELECT
    rp.channel_id,
    ROW_NUMBER() OVER(PARTITION BY rp.channel_id, rp.thread_ts ORDER BY rp.thread_ts, rp.ts, rp.channel_id) AS reply_num,
    rp.ts AS reply_ts, rp.talk_id AS reply_id, rp.talk_user AS reply_user,
    tm.thread_ts, tm.talk_id AS talk_id, tm.talk_user AS talk_user, rp.target_date
  FROM 
    reply_talk_data rp
      LEFT JOIN
        thread_first_user_master tm
      ON
        rp.channel_id = tm.channel_id AND rp.thread_ts = tm.thread_ts
  ORDER BY
    rp.thread_ts, rp.ts, rp.channel_id
),
-- reply_tableここまで
reply_thread AS (
  SELECT *
  FROM reply_table
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    talk_id = target_talk_id
),
user_table_one_day AS (
  SELECT
    user_id, real_name_normalized, display_name_normalized
  FROM
    mgm.user
  WHERE target_date ="2020-01-28"
),
channel_table_one_day AS (
  SELECT
    id, name
  FROM
    mgm.channel
  WHERE target_date ="2020-01-28"
),
one_thread_talk AS (
  SELECT
    channel_id, talk_id, talk_user, text, ts
  FROM
    mgm.talk
  WHERE
    target_date >= start_date AND target_date <= end_date AND
    talk_id = target_talk_id

  UNION ALL

  SELECT
    t.channel_id, t.talk_id, t.talk_user, t.text, t.ts
  FROM
    reply_thread r
    LEFT JOIN
      mgm.talk t
    ON
      r.reply_id = t.talk_id
  WHERE
    t.target_date >= start_date AND t.target_date <= end_date

),
thread_talks AS (
  SELECT o.channel_id, o.talk_id, o.text, o.ts, o.talk_user, u.real_name_normalized, u.display_name_normalized
  FROM one_thread_talk o
    LEFT JOIN
      user_table_one_day u
    ON o.talk_user = u.user_id
)
SELECT
  t.channel_id, c.name, t.talk_id, t.text, t.ts, t.talk_user, t.real_name_normalized, t.display_name_normalized
FROM thread_talks t
  LEFT JOIN channel_table_one_day c
  ON t.channel_id = c.id
ORDER BY ts 
;

Executing query with job ID: 958a7ec7-4571-49eb-8f86-98ff01dbe5f3
Query executing: 0.33s

In [ ]:
df_longest_reply

In [60]:
df_longest_reply.to_csv('longest-2.csv')

In [20]:
%%bigquery --project {pjt_code} df_talks
SELECT *
FROM mgm.talk t
  LEFT JOIN mgm.user u
  ON t.channel_id = u.

,channel_id,talk_id,ts,thread_ts,talk_user,text,target_date
0,CLEQVN58F,8e574373-0e15-4330-901f-a62e53797d4c,2019-07-17 09:32:18.000200+00:00,None,None,New lines on <https://scrapbox.io/dl-guild/|dl...,2019-07-18
1,CLEQVN58F,8cc7433e-0cc8-41bc-9e2f-a74f8d1a4f6e,2019-07-17 09:26:43.000100+00:00,None,None,New lines on <https://scrapbox.io/dl-guild/|dl...,2019-07-18
2,CLEQVN58F,18e2a418-359d-4d6d-bb71-ccaffa39cacc,2019-07-17 08:45:56.000100+00:00,None,None,New lines on <https://scrapbox.io/dl-guild/|dl...,2019-07-18
3,CLEQVN58F,cb135063-bdf5-4747-b180-eb7cc2c2993e,2019-07-17 08:35:38.000300+00:00,None,None,New lines on <https://scrapbox.io/dl-guild/|dl...,2019-07-18
4,CLEQVN58F,e427e79d-8ba6-404b-a22e-f8cecbc019ab,2019-07-17 08:35:01.000200+00:00,None,None,New lines on <https://scrapbox.io/dl-guild/|dl...,2019-07-18
5,CLEQVN58F,7a47a78c-9984-4a9b-bacf-3d591950c531,2019-07-17 08:29:54.000100+00:00,None,None,New lines on <https://scrapbox.io/dl-guild/|dl...,2019-07-18
6,CJCN9CH43,f38663f7-38a6-484f-9141-63b143cefaa4,2019-07-17 11:55:26.000600+00:00,None,UJRAL005U,<https://note.mu/hik0107/n/n24d682041d1b>,2019-07-18
7,CJQ8QNGBF,f67c98d9-9b3d-418f-bc76-aeee64acf147,2019-07-17 11:42:09.006100+00:00,None,UJRAL005U,【分析設計スレッド】\n<@UL2TSDLG6> さんを中心に展開,2019-07-18
8,CJQ8QNGBF,c3955a21-d4a2-42f9-bc0d-0f3f9db77a41,2019-07-17 11:40:57.005300+00:00,None,UJRAL005U,【開発関連スレッド】\n全員が担当,2019-07-18
9,CJQ8QNGBF,73ddc1b1-096c-409c-bd7c-1714edbf702d,2019-07-17 11:39:57.003700+00:00,None,UJRAL005U,【note関連スレッド】\n主に<@UKT3G90U8> さん <@UL02Y5QAY> さ...,2019-07-18


In [21]:
%%bigquery --project {pjt_code} df_u
select user_id, real_name_normalized, display_name_normalized
from mgm.user
where target_date ="2020-01-28"


In [22]:
df_u

,user_id,real_name_normalized,display_name_normalized,target_date
0,UKFMDBUV9,小池達朗,None,2020-01-01
1,UKFMDBUV9,小池達朗,None,2020-01-01
2,UKG6BDAPL,鈴木裕太,None,2020-01-01
3,UKG6BDAPL,鈴木裕太,None,2020-01-01
4,UKHMYQBEE,hiromu auchi,None,2020-01-01
5,UKHMYQBEE,hiromu auchi,None,2020-01-01
6,UKKMSRWJV,greenmidori83,None,2020-01-01
7,UKKMSRWJV,greenmidori83,None,2020-01-01
8,UKLQEKEV7,本間 翔太,None,2020-01-01
9,UKLQEKEV7,本間 翔太,None,2020-01-01
